In [1]:
import pandas as pd
import numpy as np
import os
import json   
import statsmodels.api as sm

In [2]:
base_dir = '/Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/'
index_cons_with_risk_metrics_file = 'nifty50_cons_with_risk_metrics.csv'
index_values_file = 'nifty50_index_prices_2021_2025.csv'

In [3]:
price_data = pd.read_csv(base_dir + 'all_price_data.csv', parse_dates=['Date'])
price_data['daily_return'] = price_data.groupby('Symbol')['Close'].pct_change()
price_data.dropna(inplace=True)
price_data

,Symbol,Date,Close,Volume,daily_return
1,BHARTIARTL,2020-01-02,435.132721,4933053,0.004192
2,BHARTIARTL,2020-01-03,435.037109,5154587,-0.000220
3,BHARTIARTL,2020-01-06,429.827362,7538915,-0.011975
4,BHARTIARTL,2020-01-07,425.477997,4353883,-0.010119
5,BHARTIARTL,2020-01-08,438.621796,14118818,0.030892
...,...,...,...,...,...
84852,M&M,2025-12-24,3636.699951,842897,0.003200
84853,M&M,2025-12-25,3636.699951,0,0.000000
84854,M&M,2025-12-26,3623.100098,829910,-0.003740
84855,M&M,2025-12-29,3592.100098,1034692,-0.008556


In [4]:
index_values = pd.read_csv(os.path.join(base_dir, index_values_file))
index_values

,Date,Adj Close,Close,High,Low,Open,Volume
0,2020-01-01,12182.500000,12182.500000,12222.200195,12165.299805,12202.150391,304100
1,2020-01-02,12282.200195,12282.200195,12289.900391,12195.250000,12198.549805,407700
2,2020-01-03,12226.650391,12226.650391,12265.599609,12191.349609,12261.099609,428800
3,2020-01-06,11993.049805,11993.049805,12179.099609,11974.200195,12170.599609,396500
4,2020-01-07,12052.950195,12052.950195,12152.150391,12005.349609,12079.099609,447800
...,...,...,...,...,...,...,...
1480,2025-12-23,26177.150391,26177.150391,26233.550781,26119.050781,26205.199219,216600
1481,2025-12-24,26142.099609,26142.099609,26236.400391,26123.000000,26170.650391,188800
1482,2025-12-26,26042.300781,26042.300781,26144.199219,26008.599609,26121.250000,142200
1483,2025-12-29,25942.099609,25942.099609,26106.800781,25920.300781,26063.349609,234300


In [5]:
index_cons_with_risk_metrics = pd.read_csv(os.path.join(base_dir, index_cons_with_risk_metrics_file))
index_cons_with_risk_metrics

,Date,Index_name,Security Symbol,ff_mcap,Beta,Volatility
0,2021-03-31,Nifty 50,ADANIPORTS,51375.69,0.783770,0.363451
1,2021-03-31,Nifty 50,JSWSTEEL,45293.88,1.224322,0.407596
2,2021-03-31,Nifty 50,KOTAKBANK,257059.94,1.126644,0.387015
3,2021-03-31,Nifty 50,LT,171376.76,0.853390,0.328698
4,2021-03-31,Nifty 50,M&M,76125.96,1.264854,0.453830
...,...,...,...,...,...,...
479,2025-09-30,Nifty 50,HINDALCO,NaN,1.216176,0.277803
480,2025-09-30,Nifty 50,HINDUNILVR,NaN,0.441855,0.195020
481,2025-09-30,Nifty 50,ICICIBANK,NaN,0.880423,0.173273
482,2025-09-30,Nifty 50,COALINDIA,NaN,0.933042,0.213540


Add Signal to noise ratio

In [6]:
index_narrowing_1 = index_cons_with_risk_metrics.copy()

In [7]:
index_narrowing_1['SNR'] = index_narrowing_1['Beta'] / index_narrowing_1['Volatility']**2
index_narrowing_1['rank_snr'] = index_narrowing_1['SNR'].groupby(index_narrowing_1['Date']).rank(ascending=False)
index_narrowing_1

,Date,Index_name,Security Symbol,ff_mcap,Beta,Volatility,SNR,rank_snr
0,2021-03-31,Nifty 50,ADANIPORTS,51375.69,0.783770,0.363451,5.933296,34.0
1,2021-03-31,Nifty 50,JSWSTEEL,45293.88,1.224322,0.407596,7.369462,15.0
2,2021-03-31,Nifty 50,KOTAKBANK,257059.94,1.126644,0.387015,7.521950,12.0
3,2021-03-31,Nifty 50,LT,171376.76,0.853390,0.328698,7.898641,6.0
4,2021-03-31,Nifty 50,M&M,76125.96,1.264854,0.453830,6.141214,31.0
...,...,...,...,...,...,...,...,...
479,2025-09-30,Nifty 50,HINDALCO,NaN,1.216176,0.277803,15.758734,31.0
480,2025-09-30,Nifty 50,HINDUNILVR,NaN,0.441855,0.195020,11.617744,45.0
481,2025-09-30,Nifty 50,ICICIBANK,NaN,0.880423,0.173273,29.324557,1.0
482,2025-09-30,Nifty 50,COALINDIA,NaN,0.933042,0.213540,20.461674,8.0


In [8]:
# ------------------------------------------------------------
# CONFIGURATION
# ------------------------------------------------------------

TC_COST = 0.002      # 20 bps transaction cost
MIN_OBS = 200        # minimum observations for estimation

In [9]:
# ------------------------------------------------------------
# PREPARE BENCHMARK RETURNS
# ------------------------------------------------------------
benchmark_returns = index_values.copy()
benchmark_returns["benchmark_return"] = benchmark_returns['Adj Close'].pct_change()
benchmark_returns['Date'] = pd.to_datetime(benchmark_returns['Date'])
benchmark_returns = benchmark_returns[['Date', 'benchmark_return' ]].set_index("Date").dropna()


benchmark_returns

,benchmark_return
Date,
2020-01-02,0.008184
2020-01-03,-0.004523
2020-01-06,-0.019106
2020-01-07,0.004995
2020-01-08,-0.002290
...,...
2025-12-23,0.000181
2025-12-24,-0.001339
2025-12-26,-0.003818


In [10]:
# ------------------------------------------------------------
# PREPARE STOCK RETURN PANEL
# ------------------------------------------------------------

stock_returns = price_data.pivot(
    index="Date",
    columns="Symbol",
    values="daily_return"
)
stock_returns

Symbol,ADANIENT,ADANIPORTS,APOLLOHOSP,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJAJFINSV,BAJFINANCE,BEL,BHARTIARTL,...,TATACONSUM,TATASTEEL,TCS,TECHM,TITAN,TMCV,TRENT,ULTRACEMCO,UPL,WIPRO
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,0.016117,0.014564,0.047884,-0.001422,0.011019,-0.009238,0.013558,0.003486,0.029955,0.004192,...,-0.001544,0.036558,-0.004590,0.005183,0.000736,NaN,0.022090,0.044231,0.012240,0.002422
2020-01-03,-0.013731,-0.001696,-0.005721,-0.021919,-0.018495,-0.015684,-0.016806,-0.012388,-0.021328,-0.000220,...,-0.015927,-0.002372,0.019929,0.011814,-0.013759,NaN,-0.010067,-0.006031,-0.007893,0.011277
2020-01-06,-0.042007,-0.006013,-0.015880,-0.025265,-0.026516,-0.011198,-0.032437,-0.046918,-0.039624,-0.011975,...,-0.019170,-0.021604,-0.000091,-0.006064,0.016583,NaN,-0.011103,-0.014718,-0.009818,0.004182
2020-01-07,0.022551,0.012099,0.011077,0.010105,0.003457,0.000099,0.005856,0.002740,-0.006189,-0.010119,...,0.008971,0.006022,0.002454,0.008697,0.001165,NaN,0.005661,0.020447,0.017266,0.012096
2020-01-08,-0.012497,0.002209,-0.003415,0.002552,-0.001722,0.007011,0.005481,0.010555,-0.009341,0.030892,...,0.004287,-0.001785,0.022395,-0.009394,-0.014354,NaN,-0.001877,0.018116,0.009747,-0.001568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-24,-0.011606,0.000469,0.013925,-0.007942,0.001061,0.007803,-0.005812,0.000297,0.001502,0.000377,...,-0.005398,-0.004857,0.002719,-0.000919,-0.004203,-0.040718,0.023918,0.007364,-0.010627,-0.012307
2025-12-25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2025-12-26,0.003239,-0.004818,-0.002231,-0.014001,0.001549,-0.011505,-0.008891,-0.011565,-0.003875,-0.008617,...,-0.002968,-0.005586,-0.011751,-0.011768,0.021155,-0.000245,-0.001002,0.002550,0.001747,-0.006566


In [11]:
stock_returns.fillna(method='ffill', inplace=True)

/var/folders/7g/dzc0yw6s4h19dccysb1yd3rc0000gn/T/ipykernel_42229/931075807.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock_returns.fillna(method='ffill', inplace=True)


In [12]:
stock_returns.index

DatetimeIndex(['2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07',
               '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13',
               '2020-01-14', '2020-01-15',
               ...
               '2025-12-17', '2025-12-18', '2025-12-19', '2025-12-22',
               '2025-12-23', '2025-12-24', '2025-12-25', '2025-12-26',
               '2025-12-29', '2025-12-30'],
              dtype='datetime64[ns]', name='Date', length=1487, freq=None)

In [13]:
stock_returns.index.intersection(pd.to_datetime(benchmark_returns.index))

DatetimeIndex(['2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07',
               '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13',
               '2020-01-14', '2020-01-15',
               ...
               '2025-12-16', '2025-12-17', '2025-12-18', '2025-12-19',
               '2025-12-22', '2025-12-23', '2025-12-24', '2025-12-26',
               '2025-12-29', '2025-12-30'],
              dtype='datetime64[ns]', name='Date', length=1484, freq=None)

In [14]:
common_dates = stock_returns.index.intersection(pd.to_datetime(benchmark_returns.index))
stock_returns = stock_returns.loc[common_dates]
benchmark_returns = benchmark_returns.loc[common_dates]

In [15]:

# ------------------------------------------------------------
# HELPER FUNCTIONS
# ------------------------------------------------------------

# def compute_projection_weights(returns_df, benchmark_series):
#     """
#     Solve: min Var(r_p - r_b)
#     via OLS projection: r_b = R w + e
#     """
#     X = returns_df.values
#     y = benchmark_series.loc[returns_df.index].values

#     model = sm.OLS(y, X).fit()
#     weights = pd.Series(model.params, index=returns_df.columns)

#     return weights / weights.sum()

def compute_projection_weights(returns_df, benchmark_series):

    df = returns_df.copy()
    df["benchmark"] = benchmark_series

    # Drop rows where benchmark is missing
    df = df.dropna(subset=["benchmark"])

    # Drop stocks with too little history (optional but recommended)
    min_stock_obs = 252  # one year
    valid_stocks = [
        c for c in returns_df.columns
        if returns_df[c].notna().sum() >= min_stock_obs
    ]

    df = df[valid_stocks + ["benchmark"]]

    X = df[valid_stocks].values
    y = df["benchmark"].values

    model = sm.OLS(y, X, missing="drop").fit()
    weights = pd.Series(model.params, index=valid_stocks)

    return weights / weights.sum()



def compute_tracking_error(portfolio_returns, benchmark_series):
    if isinstance(benchmark_series, pd.DataFrame):
        benchmark_series = benchmark_series.iloc[:, 0]

    aligned_benchmark = benchmark_series.reindex(portfolio_returns.index)
    diff = portfolio_returns - aligned_benchmark
    return np.std(diff)


def compute_turnover(prev_weights, new_weights):
    """
    No transaction cost for first portfolio (prev_weights is None)
    """
    if prev_weights is None:
        return 0.0

    aligned = pd.concat([prev_weights, new_weights], axis=1).fillna(0)
    return np.sum(np.abs(aligned.iloc[:, 0] - aligned.iloc[:, 1]))



In [17]:
# ------------------------------------------------------------
# FORWARD SELECTION BY REBALANCE DATE
# ------------------------------------------------------------

# IMPORTANT: prev_rebalance_weights must live OUTSIDE the rebalance loop
prev_rebalance_weights = None

rebalance_dates = sorted(index_narrowing_1["Date"].unique())

for reb_date in rebalance_dates:

    reb_date_dt = pd.to_datetime(reb_date)
    print(f"\nRunning narrowing for rebalance: {reb_date_dt.date()}")

    # ---- Slice ranking for this rebalance
    narrowing_slice = index_narrowing_1[
        index_narrowing_1["Date"] == reb_date
    ]

    ranked_symbols = (
        narrowing_slice
        .sort_values("rank_snr")
        ["Security Symbol"]
        .tolist()
    )

    # ---- Returns available up to rebalance date
    stock_ret_reb = stock_returns.loc[:reb_date]
    bench_ret_reb = benchmark_returns.loc[:reb_date]

    # ---- Initialize for this rebalance
    selected_symbols = []
    prev_net_utility = None
    best_weights = None
    best_k = None

    all_results = []

    for k, symbol in enumerate(ranked_symbols, start=1):

        # Step 2: expand S(k)
        selected_symbols.append(symbol)

        # Align to benchmark calendar (do NOT drop rows across stocks)
        returns_k = stock_ret_reb[selected_symbols]
        returns_k = returns_k.loc[bench_ret_reb.index]

        # Require sufficient benchmark history only
        if len(bench_ret_reb) < MIN_OBS:
            continue

        # Step 3: optimal tracking weights
        weights = compute_projection_weights(returns_k, bench_ret_reb)

        portfolio_returns = returns_k @ weights

        # Step 4: tracking error
        te = compute_tracking_error(portfolio_returns, bench_ret_reb)

        # Step 4: transaction cost
        # TC is evaluated vs PREVIOUS REBALANCE portfolio only
        if prev_rebalance_weights is None:
            tc = 0.0
        else:
            aligned = pd.concat(
                [prev_rebalance_weights, weights],
                axis=1
            ).fillna(0)

            turnover = np.sum(
                np.abs(aligned.iloc[:, 0] - aligned.iloc[:, 1])
            )
            tc = TC_COST * turnover

        # Step 4: net replication utility
        net_replication_utility = -te - tc

        # ---- store portfolio-level diagnostics
        all_results.append({
            "rebalance_date": reb_date,
            "k": k,
            "tracking_error": te,
            "transaction_cost": tc,
            "net_replication_utility": net_replication_utility,
            "weights": json.dumps(weights.to_dict())
        })

        # ---- first feasible portfolio is always accepted
        if best_weights is None:
            best_weights = weights.copy()
            best_k = k
            prev_net_utility = net_replication_utility
            continue

        # ---- marginal stopping rule (THIS matches your paper)
        if net_replication_utility - prev_net_utility <= 0:
            print(f"Stopping at k = {k-1} for {reb_date_dt.date()}")
            break

        prev_net_utility = net_replication_utility
        best_weights = weights.copy()
        best_k = k

    # ---- FINALIZE this rebalance (THIS is when trading occurs)
    if best_weights is not None:
        prev_rebalance_weights = best_weights.copy()
    else:
        print(f"WARNING: no valid portfolio at {reb_date_dt.date()}, carrying forward.")

    # ------------------------------------------------------------
    # OUTPUT
    # ------------------------------------------------------------

    results_df = pd.DataFrame(all_results)

    results_df.to_csv(
        os.path.join(
            base_dir,
            "Rebalance files",
            f"forward_index_narrowing_by_rebalance_{reb_date_dt.date()}.csv"
        ),
        index=False
    )

    print("Final k*:", best_k)
    print(results_df)



Running narrowing for rebalance: 2021-03-31
Stopping at k = 23 for 2021-03-31
Final k*: 23
   rebalance_date   k  tracking_error  transaction_cost  \
0      2021-03-31   1        0.014377               0.0   
1      2021-03-31   2        0.009815               0.0   
2      2021-03-31   3        0.007145               0.0   
3      2021-03-31   4        0.006554               0.0   
4      2021-03-31   5        0.005897               0.0   
5      2021-03-31   6        0.005521               0.0   
6      2021-03-31   7        0.005489               0.0   
7      2021-03-31   8        0.005301               0.0   
8      2021-03-31   9        0.005217               0.0   
9      2021-03-31  10        0.005104               0.0   
10     2021-03-31  11        0.004422               0.0   
11     2021-03-31  12        0.004285               0.0   
12     2021-03-31  13        0.004155               0.0   
13     2021-03-31  14        0.004058               0.0   
14     2021-03-31  15  